# Création variables  count index, count siren et maximum pct intersection

La siretisation repose sur une matrice de règles de gestion classée de manière ordonnée. Pour créer la matrice, il faut au préalable créer les variables nécéssaires à la création des tests. 

Le tableau ci dessous indique l'ensemble des tests a réaliser ainsi que leur dépendence.


Lors de cette US, nous allons créer 3 variables qui vont permettre a la réalisation des tests `test_pct_intersection`,`test_index_id_duplicate` et `test_siren_insee_siren_inpi`. Les trois variables sont les suivantes:

- `count_inpi_index_id_siret`: nombre de siret possible par `index_id`
- `count_inpi_siren_siret`: nombre de siret unique par siren
- `index_id_max_intersection`: Pourcentage maximum de pct_intersection par index
    - Creation de `pct_intersection` lors de l'US [3275](https://tree.taiga.io/project/olivierlubet-air/us/3275) -> `intersection / union_ as pct_intersection`
* Il faut penser a garder la variable `row_id` 

## Objective(s)



## Metadata 

* Metadata parameters are available here: 
* US Title: Création variables  count index, count siren et maximum pct intersection
* Epic: Epic 5
* US: US 7
* Date Begin: 9/1/2020
* Duration Task: 6
* Status: Active
* Source URL: [US 07 Preparation tables et variables tests](https://coda.io/d/_dCtnoqIftTn/US-07-Preparation-tables-et-variables-tests_suFb9)
* Task type:
  * Jupyter Notebook
* Users: :
  * Thomas Pernet
* Watchers:
  * Thomas Pernet
* Estimated Log points:
  * One being a simple task, 15 a very difficult one
  *  5
* Task tag
  *  #sql-query,#regle-de-gestion,#preparation-nb-index,#preparation-nb-siren,"preparation-pct-max
* Toggl Tag
  * #data-preparation
  
## Input Cloud Storage [AWS]

If link from the internet, save it to the cloud first

### Tables [AWS]

1. Batch 1:
  * Select Provider: 
  * Select table(s): 
    * Select only tables created from the same notebook, else copy/paste selection to add new input tables
    * If table(s) does not exist, add them: Add New Table
    * Information:
      * Region: 
        * Name: 
        * Code: 
      * Database: 
      * Notebook construction file: 
    
## Destination Output/Delivery

1. AWS
    1. Athena: 
      * Region: Europe (Paris)
      * Database: siretisation
      * Tables (Add name new table): ets_insee_inpi_var_group_max
      * List new tables
      * ets_insee_inpi_var_group_max

## Things to know (Steps, Attention points or new flow of information)

### Sources of information  (meeting notes, Documentation, Query, URL)

Sources of information  (meeting notes, Documentation, Query, URL)
1. Jupyter Notebook (Github Link)
  1. md : https://github.com/thomaspernet/InseeInpi_matching/blob/master/Notebooks_matching/Data_preprocessed/programme_matching/08_US_DATUM/05_creation_table_cases.md#create-table

## Connexion serveur

In [1]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_athena import service_athena
from awsPy.aws_s3 import service_s3
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import os, shutil

path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = r"{}/credential_AWS.json".format(parent_path)
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = 'eu-west-3')

region = 'eu-west-3'
bucket = 'calfdata'

In [2]:
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = region)
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = bucket, verbose = False) 

In [3]:
pandas_setting = True
if pandas_setting:
    cm = sns.light_palette("green", as_cmap=True)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

# Input/output

In [5]:
s3_output = 'inpi/sql_output'
database = 'siretisation'

In [6]:
query = """
DROP TABLE siretisation.ets_insee_inpi_var_group_max;
"""
s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = None, ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

{'Results': {'State': 'SUCCEEDED',
  'SubmissionDateTime': datetime.datetime(2020, 9, 8, 10, 45, 4, 856000, tzinfo=tzlocal()),
  'CompletionDateTime': datetime.datetime(2020, 9, 8, 10, 45, 6, 56000, tzinfo=tzlocal())},
 'QueryID': '09cfd62f-cade-4878-a5fa-552788679026'}

In [8]:
query = """
CREATE TABLE siretisation.ets_insee_inpi_var_group_max
WITH (
  format='PARQUET'
) AS
WITH create_var AS (
SELECT 
row_id,
index_id,
siren, 
siret, 
CAST(
      cardinality(
        array_distinct(
          array_intersect(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as intersection, 
    CAST(
      cardinality(
        array_distinct(
          array_union(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as union_, 
    CAST(
      cardinality(
        array_distinct(
          array_intersect(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    )/ CAST(
      cardinality(
        array_distinct(
          array_union(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as pct_intersection
    FROM siretisation.ets_insee_inpi
)
SELECT 
row_id,
create_var.index_id,
create_var.siren,
create_var.siret,
count_inpi_index_id_siret,
count_inpi_siren_siret,
create_var.pct_intersection,
index_id_max_intersection
FROM 
    create_var
LEFT JOIN (
          SELECT 
            index_id, 
            COUNT(
              DISTINCT(siret)
            ) AS count_inpi_index_id_siret 
          FROM 
            create_var 
          GROUP BY 
            index_id
        ) AS count_rows_index_id_siret ON create_var.index_id = count_rows_index_id_siret.index_id 
        LEFT JOIN (
          SELECT 
            siren, 
            COUNT(
              DISTINCT(siret)
            ) AS count_inpi_siren_siret 
          FROM 
            create_var 
          GROUP BY 
            siren
        ) AS count_rows_sequence ON create_var.siren = count_rows_sequence.siren 
        LEFT JOIN (
          SELECT 
            index_id, 
            MAX(pct_intersection) AS index_id_max_intersection 
          FROM 
            create_var 
          GROUP BY 
            index_id
        ) AS is_index_id_index_id_max_intersection ON create_var.index_id = is_index_id_index_id_max_intersection.index_id
"""

s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = None, ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

{'Results': {'State': 'SUCCEEDED',
  'SubmissionDateTime': datetime.datetime(2020, 9, 8, 10, 46, 15, 451000, tzinfo=tzlocal()),
  'CompletionDateTime': datetime.datetime(2020, 9, 8, 10, 46, 34, 711000, tzinfo=tzlocal())},
 'QueryID': '9048239d-2017-43bb-9d3e-ea88bbfcad32'}

In [9]:
query = """
SELECT *
FROM siretisation.ets_insee_inpi_var_group_max
LIMIT 10
"""
tb = s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = 'exemple_US_max', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )
tb

,row_id,index_id,siren,siret,count_inpi_index_id_siret,count_inpi_siren_siret,pct_intersection,index_id_max_intersection
0,81006,1554818,60748845,6074884500017,1,1,1.00,1.00
1,81033,7272926,811485242,81148524200010,1,1,1.00,1.00
2,81129,1750876,309113728,30911372800021,1,1,0.75,0.75
3,81237,88595,848657367,84865736700019,1,1,1.00,1.00
4,81412,213412,848658209,84865820900012,1,1,0.60,0.60
5,81498,685929,421696428,42169642800034,1,3,1.00,1.00
6,81524,364556,848659603,84865960300015,1,1,1.00,1.00
7,81536,10974270,848664009,84866400900018,1,1,1.00,1.00
8,81939,5755825,803329358,80332935800011,1,1,1.00,1.00
9,82098,745607,309304525,30930452500053,1,1,1.00,1.00


In [ ]:
#print(tb.to_markdown())

# Test acceptance

1. Vérifier que le nombre de lignes est indentique avant et après la création des variables
2. Compter le nombre de lignes par possibilité pour `count_inpi_index_id_siret` et `count_inpi_siren_siret` -> TOP 10
3. Donner la distribution de  `pct_intersection` et `index_id_max_intersection`
    - distribution -> 0.1,0.25,0.5,0.75,0.8,0.95
4. Donner la distribution de `pct_intersection` par possibilité pour `count_inpi_index_id_siret` et `count_inpi_siren_siret`
    - distribution -> 0.1,0.25,0.5,0.75,0.8,0.95
    - TOP 10
    - BOTTOM 10
5. Donner la distribution de `index_id_max_intersection` par possibilité pour `count_inpi_index_id_siret` et `count_inpi_siren_siret`
    - distribution -> 0.1,0.25,0.5,0.75,0.8,0.95
    - TOP 10
    - BOTTOM 10

## 1. Vérifier que le nombre de lignes est indentique avant et après la création des variables

In [10]:
query = """
SELECT COUNT(*)
FROM siretisation.ets_insee_inpi_var_group_max
"""
s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'count_ets_insee_inpi', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

,_col0
0,11650040


In [11]:
query = """
SELECT COUNT(*)
FROM siretisation.ets_insee_inpi_var_group_max
"""
s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'count_ets_insee_inpi', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

,_col0
0,11650040


## 2. Compter le nombre de lignes par possibilité pour `count_inpi_index_id_siret` et `count_inpi_siren_siret`

`count_inpi_index_id_siret`

In [12]:
query = """
SELECT count_inpi_index_id_siret, COUNT(*) as cnt_count_inpi_index_id_siret
FROM siretisation.ets_insee_inpi_var_group_max
GROUP BY count_inpi_index_id_siret
ORDER BY cnt_count_inpi_index_id_siret DESC
LIMIT 10
"""
tb = s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'cnt_count_inpi_index_id_siret', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )
tb

,count_inpi_index_id_siret,cnt_count_inpi_index_id_siret
0,1,8532907
1,2,2000038
2,3,506094
3,4,173836
4,5,77360
5,6,43578
6,7,30562
7,8,24064
8,9,18396
9,10,15380


In [ ]:
#print(tb.to_markdown())

`count_inpi_siren_siret`

In [13]:
query = """
SELECT count_inpi_siren_siret, COUNT(*) as cnt_count_inpi_siren_siret
FROM siretisation.ets_insee_inpi_var_group_max 
GROUP BY count_inpi_siren_siret
ORDER BY cnt_count_inpi_siren_siret DESC
LIMIT 10
"""
tb = s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'cnt_count_inpi_siren_siret', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )
tb

,count_inpi_siren_siret,cnt_count_inpi_siren_siret
0,1,7293996
1,2,2373894
2,3,683341
3,4,254195
4,5,119831
5,6,66526
6,7,46628
7,10925,40195
8,8,31532
9,9,24909


In [ ]:
#print(tb.to_markdown())

## 3. Donner la distribution de  `pct_intersection` et `index_id_max_intersection`
    
- distribution -> 0.1,0.25,0.5,0.75,0.8,0.95

 `pct_intersection`

In [14]:
query = """
WITH dataset AS (
  
  SELECT 
  MAP(
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95],
    approx_percentile(
      pct_intersection,
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95])
    ) AS nest
    FROM siretisation.ets_insee_inpi_var_group_max 
    ) 
    
    SELECT 
    pct, 
    value AS  pct_intersection
    FROM dataset
    CROSS JOIN UNNEST(nest) as t(pct, value)
    """
s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'dist_pct_intersection', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

,pct,pct_intersection
0,0.10,0.17
1,0.25,0.50
2,0.50,1.00
3,0.75,1.00
4,0.80,1.00
5,0.95,1.00


`index_id_max_intersection`

In [15]:
query = """
WITH dataset AS (
  
  SELECT 
  MAP(
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95],
    approx_percentile(
      index_id_max_intersection,
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95])
    ) AS nest
    FROM siretisation.ets_insee_inpi_var_group_max 
    ) 
    
    SELECT 
    pct, 
    value AS  index_id_max_intersection
    FROM dataset
    CROSS JOIN UNNEST(nest) as t(pct, value)
    """
s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'dist_pct_intersection', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

,pct,index_id_max_intersection
0,0.10,0.5
1,0.25,1.0
2,0.50,1.0
3,0.75,1.0
4,0.80,1.0
5,0.95,1.0


## 4. Donner la distribution de `pct_intersection` par possibilité pour `count_inpi_index_id_siret` et `count_inpi_siren_siret`


- distribution -> 0.1,0.25,0.5,0.75,0.8,0.95
- TOP 10
- BOTTOM 10

`count_inpi_index_id_siret`

In [16]:
query = """
WITH dataset AS (
  
  SELECT 
  count_inpi_index_id_siret,
  MAP(
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95],
    approx_percentile(
      pct_intersection,
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95])
    ) AS nest
    FROM siretisation.ets_insee_inpi_var_group_max  
    GROUP BY count_inpi_index_id_siret
    ) 
    
    SELECT 
    count_inpi_index_id_siret,
    pct, 
    value AS  pct_intersection
    FROM dataset
    CROSS JOIN UNNEST(nest) as t(pct, value)
"""
output = s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'pct_count_inpi_index_id_siret', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

In [17]:
tb= output.set_index(['count_inpi_index_id_siret', 'pct']).unstack(-1).head(10)
tb

pct_intersection                             
pct                                   0.10  0.25  0.50  0.75  0.80 0.95
count_inpi_index_id_siret                                              
1                                      0.5  1.00  1.00  1.00  1.00  1.0
2                                      0.0  0.09  0.56  1.00  1.00  1.0
3                                      0.0  0.00  0.25  1.00  1.00  1.0
4                                      0.0  0.00  0.22  0.86  1.00  1.0
5                                      0.0  0.00  0.20  0.60  1.00  1.0
6                                      0.0  0.00  0.20  0.50  0.75  1.0
7                                      0.0  0.00  0.17  0.44  0.67  1.0
8                                      0.0  0.00  0.17  0.33  0.50  1.0
9                                      0.0  0.00  0.13  0.33  0.50  1.0
10                                     0.0  0.00  0.13  0.33  0.36  1.0

In [ ]:
#print(tb.to_markdown())

In [18]:
tb = output.set_index(['count_inpi_index_id_siret', 'pct']).unstack(-1).tail(10)
tb

pct_intersection                              
pct                                   0.10  0.25  0.50  0.75  0.80  0.95
count_inpi_index_id_siret                                               
69                                    0.00  0.14  0.25  0.33  0.33  1.00
73                                    0.00  0.00  0.00  1.00  1.00  1.00
84                                    0.00  0.00  0.00  0.20  0.25  1.00
86                                    0.50  0.50  1.00  1.00  1.00  1.00
94                                    0.00  0.00  0.00  0.00  0.00  0.00
97                                    0.33  0.33  1.00  1.00  1.00  1.00
98                                    0.23  0.40  0.50  0.55  0.58  0.67
119                                   0.00  0.00  0.00  0.25  0.25  1.00
139                                   0.00  0.00  0.20  0.50  0.50  0.60
390                                   0.00  0.00  0.00  0.25  0.33  1.00

In [ ]:
#print(tb.to_markdown())

`count_inpi_siren_siret`

In [19]:
query = """
WITH dataset AS (
  
  SELECT 
  count_inpi_siren_siret,
  MAP(
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95],
    approx_percentile(
      pct_intersection,
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95])
    ) AS nest
    FROM siretisation.ets_insee_inpi_var_group_max 
    GROUP BY count_inpi_siren_siret
    ) 
    
    SELECT 
    count_inpi_siren_siret,
    pct, 
    value AS  pct_intersection
    FROM dataset
    CROSS JOIN UNNEST(nest) as t(pct, value)
"""
output = s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'pct_count_inpi_siren_siret', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

In [20]:
tb= output.set_index(['count_inpi_siren_siret', 'pct']).unstack(-1).head(10)
tb

pct_intersection                           
pct                                0.10  0.25  0.50 0.75 0.80 0.95
count_inpi_siren_siret                                            
1                                   0.5  1.00  1.00  1.0  1.0  1.0
2                                   0.0  0.25  1.00  1.0  1.0  1.0
3                                   0.0  0.14  0.67  1.0  1.0  1.0
4                                   0.0  0.08  0.50  1.0  1.0  1.0
5                                   0.0  0.09  0.50  1.0  1.0  1.0
6                                   0.0  0.11  0.50  1.0  1.0  1.0
7                                   0.0  0.13  0.50  1.0  1.0  1.0
8                                   0.0  0.13  0.50  1.0  1.0  1.0
9                                   0.0  0.17  0.60  1.0  1.0  1.0
10                                  0.0  0.13  0.50  1.0  1.0  1.0

In [ ]:
#print(tb.to_markdown())

In [21]:
tb = output.set_index(['count_inpi_siren_siret', 'pct']).unstack(-1).tail(10)
tb

pct_intersection                            
pct                                0.10 0.25  0.50  0.75  0.80 0.95
count_inpi_siren_siret                                             
1912                                0.0  0.0  0.40  1.00  1.00  1.0
1970                                0.0  0.0  0.15  0.43  0.67  1.0
2180                                0.0  0.0  0.25  1.00  1.00  1.0
2409                                0.0  0.0  0.00  0.25  0.33  1.0
2731                                0.0  0.0  0.00  0.33  0.33  1.0
2772                                0.0  0.0  0.00  0.25  0.33  1.0
2863                                0.0  0.0  0.14  0.33  0.33  1.0
3144                                0.0  0.0  0.00  0.25  0.33  1.0
3191                                0.0  0.0  0.00  0.25  0.33  1.0
10925                               0.0  0.0  0.20  1.00  1.00  1.0

In [ ]:
#print(tb.to_markdown())

## 5. Donner la distribution de `index_id_max_intersection` par possibilité pour `count_inpi_index_id_siret` et `count_inpi_siren_siret`

- distribution -> 0.1,0.25,0.5,0.75,0.8,0.95
- TOP 10
- BOTTOM 10

`count_inpi_index_id_siret`

In [ ]:
query = """
WITH dataset AS (
  
  SELECT 
  count_inpi_index_id_siret,
  MAP(
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95],
    approx_percentile(
      index_id_max_intersection,
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95])
    ) AS nest
    FROM temp_us_max 
    GROUP BY count_inpi_index_id_siret
    ) 
    
    SELECT 
    count_inpi_index_id_siret,
    pct, 
    value AS  pct_intersection
    FROM dataset
    CROSS JOIN UNNEST(nest) as t(pct, value)
"""
output = s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'pct_count_inpi_index_id_siret', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

In [ ]:
tb = output.set_index(['count_inpi_index_id_siret', 'pct']).unstack(-1).head(10)
tb

In [ ]:
#print(tb.to_markdown())

In [ ]:
tb = output.set_index(['count_inpi_index_id_siret', 'pct']).unstack(-1).tail(10)
tb

In [ ]:
#print(tb.to_markdown())

`count_inpi_siren_siret`

In [ ]:
query = """
WITH dataset AS (
  
  SELECT 
  count_inpi_siren_siret,
  MAP(
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95],
    approx_percentile(
      index_id_max_intersection,
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95])
    ) AS nest
    FROM temp_us_max 
    GROUP BY count_inpi_siren_siret
    ) 
    
    SELECT 
    count_inpi_siren_siret,
    pct, 
    value AS pct_intersection
    FROM dataset
    CROSS JOIN UNNEST(nest) as t(pct, value)
"""
output = s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'pct_count_inpi_siren_siret', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

In [ ]:
tb = output.set_index(['count_inpi_siren_siret', 'pct']).unstack(-1).head(10)
tb

In [ ]:
#print(tb.to_markdown())

In [ ]:
tb = output.set_index(['count_inpi_siren_siret', 'pct']).unstack(-1).tail(10)
tb

In [ ]:
#print(tb.to_markdown())

# Generation report

In [ ]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp

In [ ]:
def create_report(extension = "html", keep_code = False):
    """
    Create a report from the current notebook and save it in the 
    Report folder (Parent-> child directory)
    
    1. Exctract the current notbook name
    2. Convert the Notebook 
    3. Move the newly created report
    
    Args:
    extension: string. Can be "html", "pdf", "md"
    
    
    """
    
    ### Get notebook name
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[0].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+ \
                                             'api/sessions?token=' + \
                                             srv['token'])
            sessions = json.load(req)
            notebookname = sessions[0]['name']
        except:
            pass  
    
    sep = '.'
    path = os.getcwd()
    #parent_path = str(Path(path).parent)
    
    ### Path report
    #path_report = "{}/Reports".format(parent_path)
    #path_report = "{}/Reports".format(path)
    
    ### Path destination
    name_no_extension = notebookname.split(sep, 1)[0]
    source_to_move = name_no_extension +'.{}'.format(extension)
    dest = os.path.join(path,'Reports', source_to_move)
    
    ### Generate notebook
    if keep_code:
        os.system('jupyter nbconvert --to {} {}'.format(
    extension,notebookname))
    else:
        os.system('jupyter nbconvert --no-input --to {} {}'.format(
    extension,notebookname))
    
    ### Move notebook to report folder
    #time.sleep(5)
    shutil.move(source_to_move, dest)
    print("Report Available at this adress:\n {}".format(dest))

In [ ]:
create_report(extension = "html")